# MultiREx - Quickstart
#### Planetary transmission spectra generator
##### [GitHub Repository](https://github.com/D4san/MultiREx-public)

In [2]:
import multirex as mrex

Loading MultiREx version 0.1.5


### To Do 

Here, we want to show the features we should include in future versions of `MultiREx`:

#### Avanced generators 

In the present version (0.2.x), we generate system by randomly choose values from a given range. For instance:

In [3]:
system=mrex.System(
    star=mrex.Star(
        temperature=(4000,6000),
        radius=(0.5,1.5),
        mass=(0.8,1.2),
    ),
    planet=mrex.Planet(
        radius=(0.5,1.5),
        mass=(0.8,1.2),
        atmosphere=mrex.Atmosphere(
            temperature=(290,310), # in K
            base_pressure=(1e5,10e5), # in Pa
            top_pressure=(1,10), # in Pa
            fill_gas="N2", # the gas that fills the atmosphere
            composition=dict(
                CO2=(-5,-4), # This is the log10(mix-ratio)
                CH4=(-6,-5),
            )
        )
    ),
    sma=(0.5,1)
)

But it assumes that the parameters are physically and statistically independent which is obviously untrue. We may think about three type of relationship among the parameters:

- **Independence**: parameters are completely independent of other parameters. 
  - *Example*: Almost any property is completely independent but we can assume absolute independence.
  - *Specification*: In this case the specification can be provided as a number, as a range (tuple) or as a list. 

- **Combinatorial dependence**: we want that parameters be combined using specific combinations.
  - *Example*: Let's suppose that we want to create combinations of stellar masses [0.8,1.0,1.2] and planetary masses [1,5]. Combinatorial relationship means that we want to test all combinations in the cartesian product of this list of values: [0.8,1.0,1.2] x [1,5] = (0.8,1), (0.8,5), (1.0,1), etc.
  - *Specification*: In this case the values should be provided as a list.

- **Physical dependence**: when one parameter depend on other parameters due to a physical law.
  - *Example*: Stellar mass and stellar radius are physically dependent for main sequence stars.
  - *Specification*: In that case one have to provide a function to obtain a property from other properties with which it is related.

- **Statistical dependence**: when the dependency arises from correlations.
  - *Example*: It is hard to find an example, but let's assume that it has been observed that low mass stars, tend to have low mass planets. This is a statistical correlation.
  - *Specification*: A statistical dependence is specified by a covariance matrix.

This simple case before correspond to that when all parameters are completely independent. But what happen if we want to study many systems changin the mass of the star which is physically dependent on the radius of the star. The command should be:

In [6]:
def temperature_from_mass(system):
    mass = system.star.mass
    return 5000*mass

def radius_from_mass(system):
    mass = system.star.mass
    return 1.2*mass

system=mrex.System(
    star=mrex.Star(
        mass=(0.8,1.2),
        temperature=dict(value=None,func=temperature_from_mass),
        radius=dict(value=None,func=radius_from_mass),
    ),
    planet=mrex.Planet(
        radius=1.0,
        mass=1.0,
        atmosphere=mrex.Atmosphere(
            temperature=290, # in K
            base_pressure=1e5, # in Pa
            top_pressure=1, # in Pa
            fill_gas="N2", # the gas that fills the atmosphere
            composition=dict(
                CO2=-4, # This is the log10(mix-ratio)
                CH4=-6,
            )
        )
    ),
    sma=(0.5,1)
)

In this case the system should use the routine `temperature_from_mass` and `radius_from_mass` for generating the property.

In the case of statistical dependence:

In [ ]:
def multinormal(mus,sigmas):
    pass

mus = [0.8,1.0]	
sigmas = [[0.1,0.02],[0.02,0.1]]

system=mrex.System(
    star=mrex.Star(
        mass=dict(value=(mus,sigmas),func=multinormal),
        temperature=temperature_from_mass,
        radius=radius_from_mass,
    ),
    planet=mrex.Planet(
        mass=dict(value=(mus,sigmas),func=multinormal),
        radius=1.0,
        atmosphere=mrex.Atmosphere(
            temperature=290, # in K
            base_pressure=1e5, # in Pa
            top_pressure=1, # in Pa
            fill_gas="N2", # the gas that fills the atmosphere
            composition=dict(
                CO2=-4, # This is the log10(mix-ratio)
                CH4=-6,
            )
        )
    ),
    sma=(0.5,1)
)

In the case of combinatorial:

In [ ]:
def combinatorial_star_planet(system):
    star_masses = system.star._mass
    planet_masses = system.planet._mass
    return [star_masses[0],planet_masses[0]]

system=mrex.System(
    star=mrex.Star(
        mass=dict(value=[0.8,1.0,1.2],func=combinatorial_star_planet),
        temperature=temperature_from_mass,
        radius=radius_from_mass,
    ),
    planet=mrex.Planet(
        mass=dict(value=[1,5],func=combinatorial_star_planet),
        radius=1.0,
        atmosphere=mrex.Atmosphere(
            temperature=290, # in K
            base_pressure=1e5, # in Pa
            top_pressure=1, # in Pa
            fill_gas="N2", # the gas that fills the atmosphere
            composition=dict(
                CO2=-4, # This is the log10(mix-ratio)
                CH4=-6,
            )
        )
    ),
    sma=(0.5,1)
)

Unifying languages, this is the way to indicate independent values:

In [ ]:
system=mrex.System(
    star=mrex.Star(
        temperature=dict(value=(4000,6000),func=None),
        radius=dict(value=(0.5,1.5),func=None),
        mass=dict(value=(0.8,1.2),func=None),
    ),
    planet=mrex.Planet(
        radius=dict(value=(0.5,1.5),func=None),
        mass=dict(value=(0.8,1.2),func=None),
        atmosphere=mrex.Atmosphere(
            temperature=dict(value=(290,310),func=None), # in K
            base_pressure=dict(value=(1e5,10e5),func=None), # in Pa
            top_pressure=dict(value=(1,10),func=None), # in Pa
            fill_gas="N2", # the gas that fills the atmosphere
            composition=dict(
                CO2=dict(value=(-5,-4),func=None), # This is the log10(mix-ratio)
                CH4=dict(value=(-6,-5),func=None),
            )
        )
    ),
    sma=(0.5,1)
)